# Logistics Fact Table Pipeline

In [7]:
#Import libraries
import sqlite3
import json
import pandas as pd
import requests

In [8]:
#Create a connection object to run commands
conn = sqlite3.connect('new_db.db')
cursor = conn.cursor()

In [9]:
Profitability = pd.read_excel('FINAL DB.xlsx', sheet_name = 0)

In [10]:
Logistics = pd.read_excel('FINAL DB.xlsx', sheet_name = 1)

In [11]:
#Create Logistics Fact Table & specify data types and constraints
create_table_query = """ CREATE TABLE  Logistic_Fact_Table
(Order_ID NUMERIC PRIMARY KEY,
Customer_ID NUMERIC NOT NULL,
Tracking_ID NUMERIC NOT NULL,
Area_Code NUMERIC NOT NULL,
Country NUMERIC NOT NULL, 
City TEXT NOT NULL, 
Storage_Quantity NUMERIC NOT NULL)"""

In [12]:
#Execute query and make change permanent
cursor.execute(create_table_query)
conn.commit()

In [13]:
#Create all dimensions for Logistics Fact Table:

#Create Customer Returns Dimension table connected to Logistics Fact Table
create_returns_dimension = """ CREATE TABLE  returns_dimension
(Order_ID NUMERIC PRIMARY KEY,
Product_ID NUMERIC NOT NULL,
Status TEXT NOT NULL,
Return_Date TEXT NOT NULL,
Amount INTEGER NOT NULL,
Reason_for_Return TEXT NOT NULL)"""

#Execute query and make change permanent
cursor.execute(create_returns_dimension)  
conn.commit() 

In [14]:
#Create Order dimension table connected to Logistics Fact Table
create_order_dimension = """ CREATE TABLE order_dimension
(Order_ID NUMERIC PRIMARY KEY,
Product_ID NUMERIC NOT NULL,
Customer_ID NUMERIC NOT NULL,
Order_place_Date NUMERIC NOT NULL, 
Return_Status TEXT NOT NULL,
Delivery_Status TEXT NOT NULL)"""

#Execute query and make change permanent
cursor.execute(create_order_dimension)  
conn.commit() 

In [15]:
#Create Inventory Management dimension table connected to Logistics Fact Table
create_inventory_dimension = """ CREATE TABLE  inventory_dimension
(Order_ID NUMERIC PRIMARY KEY,
Date_Received TEXT NOT NULL,
Quantity NUMERIC NOT NULL,
Shipment_Date NUMERIC NOT NULL,
Shipment_Quantity NUMERIC NOT NULL,
Inspection_Date NUMERIC NOT NULL,
Quantity_Inspected NUMERIC NOT NULL)"""

#Execute query and make change permanent
cursor.execute(create_inventory_dimension)  
conn.commit() 

In [16]:
#Create Tracking dimension table connected to Logistics Fact Table
create_tracking_dimension = """ CREATE TABLE  tracking_dimension
(Tracking_ID NUMERIC PRIMARY KEY,
Warehouse_ID NUMERIC NOT NULL,
Customer_Address TEXT NOT NULL,
Shipping_Mode TEXT NOT NULL,
Order_Pickup_Date NUMERIC NOT NULL,
Delivery_Date NUMERIC NOT NULL,
Package_Weight NUMERIC NOT NULL,
Shipping_Cost NUMERIC NOT NULL,
Delivery_Status TEXT NOT NULL)"""

#Execute query and make change permanent
cursor.execute(create_tracking_dimension) 
conn.commit() 

In [17]:
#Create Shipping dimension table connected to Logistics Fact Table
create_shipping_dimension = """ CREATE TABLE  shipping_dimension
(Tracking_ID NUMERIC PRIMARY KEY,
Shipping_ID NUMERIC NOT NULL,
Carrier_ID NUMERIC NOT NULL,
Damage_Status TEXT NOT NULL,
Quantity_of_Spoilage NUMERIC NOT NULL)"""

#Execute query and make change permanent
cursor.execute(create_shipping_dimension)
conn.commit()   

In [18]:
#Create Carrier dimension table connected to Logistics Fact Table
create_carrier_dimension = """ CREATE TABLE  carrier_dimension
(Carrier_ID NUMERIC PRIMARY KEY,
Carrier_Name TEXT NOT NULL,
Carrier_Address TEXT NOT NULL,
Carrier_City TEXT NOT NULL,
Carrier_Province TEXT NOT NULL,
Carrier_Country TEXT NOT NULL,
Post_Code OBJECT NOT NULL,
Phone_Number NUMERIC NOT NULL)"""

#Execute query and make change permanent
cursor.execute(create_carrier_dimension)  
conn.commit() 

In [19]:
#Create Warehouse dimension table connected to Logistics Fact Table
create_warehouse_dimension = """ CREATE TABLE  warehouse_dimension
(Warehouse_ID NUMERIC PRIMARY KEY,
Warehouse_Name TEXT NOT NULL,
Manager TEXT NOT NULL,
Size NUMERIC NOT NULL,
Type TEXT NOT NULL,
Region TEXT NOT NULL,
Address TEXT NOT NULL,
Row NUMERIC NOT NULL,
Shell_number NUMERIC NOT NULL)"""

#Execute query and make change permanent
cursor.execute(create_warehouse_dimension)  
conn.commit() 

In [20]:
Logistic_Fact_Table = Logistics[['Order_ID', 'Customer_ID', 'Tracking_ID', 'Area_code', 'Country', 'City', 'Storage_Quantity']]

Logistic_Fact_Table['Order_ID'] = Logistic_Fact_Table['Order_ID'].astype(str) 
Logistic_Fact_Table['Customer_ID'] = Logistic_Fact_Table['Customer_ID'].astype(str) 
Logistic_Fact_Table['Tracking_ID'] = Logistic_Fact_Table['Tracking_ID'].astype(str) 
Logistic_Fact_Table['Area_code'] = Logistic_Fact_Table['Area_code'].astype(str) 
Logistic_Fact_Table['Country'] = Logistic_Fact_Table['Country'].astype(str) 
Logistic_Fact_Table['City'] = Logistic_Fact_Table['City'].astype(str) 
Logistic_Fact_Table['Storage_Quantity'] = Logistic_Fact_Table['Storage_Quantity'].astype(str) 

/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/3509309299.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Logistic_Fact_Table['Order_ID'] = Logistic_Fact_Table['Order_ID'].astype(str)
/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/3509309299.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Logistic_Fact_Table['Customer_ID'] = Logistic_Fact_Table['Customer_ID'].astype(str)
/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/3509309299.py:5: SettingWithC

In [21]:
def insert_records(recordList):
    insert_record_query = """ INSERT INTO Logistic_Fact_Table (Order_ID, Customer_ID, Tracking_ID, Area_Code, Country, City, Storage_Quantity) VALUES (?, ?,?,?,?,?,?)"""
    cursor.executemany(insert_record_query, recordList)
    conn.commit()
    print('New data inserted successfully')

#Insert records in a list
insert_list = Logistic_Fact_Table.to_records(index=False)
insert_records(insert_list)

New data inserted successfully


In [22]:
returns_dimension = Logistics[['Order_ID', 'Product_ID', 'Status', 'Return_Date', 'Amount', 'Reason_for_Return']]
returns_dimension['Order_ID'] = returns_dimension['Order_ID'].astype(str) 
returns_dimension['Product_ID'] = returns_dimension['Product_ID'].astype(str) 
returns_dimension['Status'] = returns_dimension['Status'].astype(str) 
returns_dimension['Return_Date'] = returns_dimension['Return_Date'].astype(str) 
returns_dimension['Amount'] = returns_dimension['Amount'].astype(str) 
returns_dimension['Reason_for_Return'] = returns_dimension['Reason_for_Return'].astype(str) 


/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/1667627323.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  returns_dimension['Order_ID'] = returns_dimension['Order_ID'].astype(str)
/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/1667627323.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  returns_dimension['Product_ID'] = returns_dimension['Product_ID'].astype(str)
/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/1667627323.py:4: SettingWithCopyWarning

In [23]:
#Create Records for Returns Dimension Table
def insert_records(recordList):
    insert_record_query2 = """ INSERT INTO returns_dimension (Order_ID, Product_ID, Status, Return_Date, Amount, Reason_for_Return) VALUES (?, ?, ?, ?, ?, ?)"""
    cursor.executemany(insert_record_query2, recordList)
    conn.commit()
    print('New data inserted successfully')
    
insert_list = returns_dimension.to_records(index=False)
insert_records(insert_list)

New data inserted successfully


In [24]:
order_dimension = Logistics[["Order_ID", 'Product_ID','Customer_ID', 'Order_place_Date', 'Return_Status', 'Delivery_Status']]
order_dimension['Order_ID']= order_dimension['Order_ID'].astype(str) 
order_dimension['Product_ID']= order_dimension['Product_ID'].astype(str)
order_dimension['Customer_ID']= order_dimension['Customer_ID'].astype(str)
order_dimension['Order_place_Date']= order_dimension['Order_place_Date'].astype(str)
order_dimension['Return_Status']= order_dimension['Return_Status'].astype(str)
order_dimension['Delivery_Status']= order_dimension['Delivery_Status'].astype(str)

/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/2688953766.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_dimension['Order_ID']= order_dimension['Order_ID'].astype(str)
/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/2688953766.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_dimension['Product_ID']= order_dimension['Product_ID'].astype(str)
/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/2688953766.py:4: SettingWithCopyWarning: 
A value

In [25]:
#Create Records for Order Dimension Table
def insert_records(recordList):
    insert_record_query2 = """ INSERT INTO order_dimension (Order_ID, Product_ID, Customer_ID, Order_place_Date, Return_Status, Delivery_Status) VALUES (?, ?,?,?,?,?)"""
    cursor.executemany(insert_record_query2, recordList)
    conn.commit()
    print('New data inserted successfully')

insert_list = order_dimension.to_records(index=False)
insert_records(insert_list)

New data inserted successfully


In [26]:
inventory_dimension = Logistics[['Order_ID', 'Date_Received', 'Quantity', 'Shipment_Date', 'Shipment_Quantity', 'Inspection_Date', 'Quantity_Inspect']]
inventory_dimension['Order_ID'] = inventory_dimension['Order_ID'].astype(str) 
inventory_dimension['Date_Received'] = inventory_dimension['Date_Received'].astype(str) 
inventory_dimension['Quantity'] = inventory_dimension['Quantity'].astype(str) 
inventory_dimension['Shipment_Date'] = inventory_dimension['Shipment_Date'].astype(str) 
inventory_dimension['Shipment_Quantity'] = inventory_dimension['Shipment_Quantity'].astype(str) 
inventory_dimension['Inspection_Date'] = inventory_dimension['Inspection_Date'].astype(str) 
inventory_dimension['Quantity_Inspect'] = inventory_dimension['Quantity_Inspect'].astype(str) 

/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/2246235975.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inventory_dimension['Order_ID'] = inventory_dimension['Order_ID'].astype(str)
/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/2246235975.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inventory_dimension['Date_Received'] = inventory_dimension['Date_Received'].astype(str)
/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/2246235975.py:4: SettingW

In [27]:
#Create Records for Inverntory Dimension Table
def insert_records(recordList):
    insert_record_query3 = """ INSERT INTO inventory_dimension (Order_ID, Date_Received, Quantity, Shipment_Date, Shipment_Quantity, Inspection_Date, Quantity_Inspected) VALUES (?,?,?,?,?,?,?)"""
    cursor.executemany(insert_record_query3, recordList)
    conn.commit()
    print('New data inserted successfully')  

insert_list = inventory_dimension.to_records(index=False)
insert_records(insert_list)

New data inserted successfully


In [28]:
shipping_dimension = Logistics[['Tracking_ID', 'Shipping_ID', 'Carrier_ID', 'Damage_Status', 'Quantity_of_Spoilage']]
shipping_dimension['Tracking_ID']=shipping_dimension['Tracking_ID'].astype(str) 
shipping_dimension['Shipping_ID']=shipping_dimension['Shipping_ID'].astype(str) 
shipping_dimension['Carrier_ID']=shipping_dimension['Carrier_ID'].astype(str) 
shipping_dimension['Damage_Status']=shipping_dimension['Damage_Status'].astype(str) 
shipping_dimension['Quantity_of_Spoilage']=shipping_dimension['Quantity_of_Spoilage'].astype(str) 


/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/1525125341.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shipping_dimension['Tracking_ID']=shipping_dimension['Tracking_ID'].astype(str)
/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/1525125341.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shipping_dimension['Shipping_ID']=shipping_dimension['Shipping_ID'].astype(str)
/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/1525125341.py:4: SettingWithCop

In [29]:
#Create Records for Shipping Dimension Table
def insert_records(recordList):
    insert_record_query4 = """ INSERT INTO shipping_dimension (Tracking_ID, Shipping_ID, Carrier_ID, Damage_Status, Quantity_of_Spoilage) VALUES (?,?,?,?, ?)"""
    cursor.executemany(insert_record_query4, recordList)
    conn.commit()
    print('New data inserted successfully')     
    

insert_list = shipping_dimension.to_records(index=False)
insert_records(insert_list)

New data inserted successfully


In [30]:
carrier_dimension = Logistics[['Carrier_ID', 'Carrier_Name', 'Carrier_Address', 'Carrier_City', 'Carrier_Province', 'Carrier_Country', 'Post_Code', 'Phone_Number']]
carrier_dimension['Carrier_ID'] = carrier_dimension['Carrier_ID'].astype(str)
carrier_dimension['Carrier_Name'] = carrier_dimension['Carrier_Name'].astype(str)
carrier_dimension['Carrier_Address'] = carrier_dimension['Carrier_Address'].astype(str)
carrier_dimension['Carrier_City'] = carrier_dimension['Carrier_City'].astype(str)
carrier_dimension['Carrier_Province'] = carrier_dimension['Carrier_Province'].astype(str)
carrier_dimension['Carrier_Country'] = carrier_dimension['Carrier_Country'].astype(str)
carrier_dimension['Post_Code'] = carrier_dimension['Post_Code'].astype(str)
carrier_dimension['Phone_Number'] = carrier_dimension['Phone_Number'].astype(str)

/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/1018048892.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  carrier_dimension['Carrier_ID'] = carrier_dimension['Carrier_ID'].astype(str)
/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/1018048892.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  carrier_dimension['Carrier_Name'] = carrier_dimension['Carrier_Name'].astype(str)
/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/1018048892.py:4: SettingWithCop

In [31]:
#Carrier Dimension Table
def insert_records(recordList):
    insert_record_query5 = """ INSERT INTO carrier_dimension (Carrier_ID, Carrier_Name, Carrier_Address, Carrier_City, Carrier_Province, Carrier_Country, Post_Code, Phone_Number) VALUES (?, ?,?,?,?,?,?,?)"""
    cursor.executemany(insert_record_query5, recordList)
    conn.commit()
    print('New data inserted successfully')     

insert_list = carrier_dimension.to_records(index=False)
insert_records(insert_list)

New data inserted successfully


In [32]:
warehouse_dimension = Logistics[['Warehouse_ID', 'Warehouse_name', 'Manager', 'Size', 'Type', 'Region', 'Address', 'Row', 'Shell_number']]
warehouse_dimension['Warehouse_ID'] = warehouse_dimension['Warehouse_ID'].astype(str)
warehouse_dimension['Warehouse_name'] = warehouse_dimension['Warehouse_name'].astype(str)
warehouse_dimension['Manager'] = warehouse_dimension['Manager'].astype(str)
warehouse_dimension['Size'] = warehouse_dimension['Size'].astype(str)
warehouse_dimension['Type'] = warehouse_dimension['Type'].astype(str)
warehouse_dimension['Region'] = warehouse_dimension['Region'].astype(str)
warehouse_dimension['Address'] = warehouse_dimension['Address'].astype(str)
warehouse_dimension['Row'] = warehouse_dimension['Row'].astype(str)
warehouse_dimension['Shell_number'] = warehouse_dimension['Shell_number'].astype(str)

/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/2483285860.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  warehouse_dimension['Warehouse_ID'] = warehouse_dimension['Warehouse_ID'].astype(str)
/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/2483285860.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  warehouse_dimension['Warehouse_name'] = warehouse_dimension['Warehouse_name'].astype(str)
/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/2483285860.py:4

In [33]:
#Warehouse Dimension Table
def insert_records(recordList):
    insert_record_query6 = """ INSERT INTO warehouse_dimension (Warehouse_ID, Warehouse_name, Manager, Size, Type, Region, Address, Row, Shell_number) VALUES (?, ?,?,?,?,?,?,?,?)"""
    cursor.executemany(insert_record_query6, recordList)
    conn.commit()
    print('New data inserted successfully')     
    
insert_list = warehouse_dimension.to_records(index=False)
insert_records(insert_list)

New data inserted successfully


# Profitability Fact Table Pipeline

In [34]:
#Create a connection object to run commands
conn = sqlite3.connect('new_db.db')
cursor = conn.cursor()
conn.commit()

In [35]:
#Create Date dimension table
create_date_dimension = """ CREATE TABLE  Date_Dimension
(Date_ID TEXT PRIMARY KEY,
Fiscal_Year VARCHAR(20) NOT NULL,
Fiscal_Quater VARCHAR(20) NOT NULL,
Fiscal_Month VARCHAR(20) NOT NULL,
Day VARCHAR(20) NOT NULL,
Time VARCHAR(20) NOT NULL,
Season TEXT NOT NULL)"""
cursor.execute(create_date_dimension)
conn.commit()

In [36]:
#Change data types 
date_dimension = Profitability[["Date_ID","Fiscal_Year","Fiscal_Quater","Fiscal_Month","Day","Time","Season"]]
date_dimension['Date_ID'] = date_dimension['Date_ID'].astype(str) 
date_dimension['Fiscal_Year'] = date_dimension['Fiscal_Year'].astype(str) 
date_dimension['Fiscal_Quater'] = date_dimension['Fiscal_Quater'].astype(str) 
date_dimension['Fiscal_Month'] = date_dimension['Fiscal_Month'].astype(str)

/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/2545110723.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_dimension['Date_ID'] = date_dimension['Date_ID'].astype(str)
/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/2545110723.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_dimension['Fiscal_Year'] = date_dimension['Fiscal_Year'].astype(str)
/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/2545110723.py:5: SettingWithCopyWarning: 
A value i

In [37]:
#Insert records within Date dimension

def insert_records(recordList):
    insert_record_query = """ INSERT INTO Date_Dimension(Date_ID,Fiscal_Year,Fiscal_Quater, Fiscal_Month,Day, Time, Season) VALUES (?, ?,?,?,?,?,?)"""
    cursor.executemany(insert_record_query, recordList)
    conn.commit()
    print('New data inserted successfully')

In [38]:
#Insert records in a list
insert_list = date_dimension.to_records(index=False)
insert_records(insert_list)

New data inserted successfully


In [39]:
#Create Currency dimension
create_currency_dimension = """ CREATE TABLE Currency_Dimension
(Currency_ID TEXT PRIMARY KEY,
Currency_Name TEXT NOT NULL,
Coversion_rate NUMERIC NOT NULL,
Country TEXT NOT NULL)"""
cursor.execute(create_currency_dimension)
conn.commit()

In [40]:
#Insert values within dimension
currency_dimension = Profitability[['Currency_ID','Coversion_rate','Country','Currency_Name']]
currency_dimension['Currency_ID'] = currency_dimension['Currency_ID'].astype(str)

/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/4203642690.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  currency_dimension['Currency_ID'] = currency_dimension['Currency_ID'].astype(str)


In [41]:
#Insert records within table
def insert_records(recordList):
    insert_record_query2 = """ INSERT INTO currency_dimension (Currency_ID,Coversion_rate,Country,Currency_Name) VALUES (?, ?,?,?)"""
    cursor.executemany(insert_record_query2, recordList)
    conn.commit()
    print('New data inserted successfully')

In [42]:
#Insert records into a list
insert_list = currency_dimension.to_records(index=False)
insert_records(insert_list) 

New data inserted successfully


In [43]:
#Create Customer Engagement table connected to profitability Fact Table
create_customer_engagement_dimension = """ CREATE TABLE  Customer_Engagement_Dimension
(Customer_ID TEXT PRIMARY KEY,
Most_frequent_purchased_product TEXT NOT NULL,
Most_frequent_purchased_category TEXT NOT NULL,
Coupon_used TEXT NOT NULL,
Customer_purchasing_power TEXT NOT NULL,
Active_Status TEXT NOT NULL,
Number_of_Clicks Varchar(20) NOT NULL,
Coversion_rate Varchar(20) NOT NULL,
Average_Day_Between_Purchase Varchar(20) NOT NULL)"""
cursor.execute(create_customer_engagement_dimension)
conn.commit()

In [44]:
#Create Customer Engagement dimensions 
customer_engagement_dimension = Profitability[['Customer_ID','Most_frequent_purchased_product','Most_frequent_purchased_category','Coupon_used','Customer_purchasing_power','Active_Status','Number_of_Clicks','Coversion_rate','Average_Day_Between_Purchase']]

In [45]:
#Add dimensions
customer_engagement_dimension['Customer_ID'] = customer_engagement_dimension['Customer_ID'].astype(str)
customer_engagement_dimension['Active_Status'] = customer_engagement_dimension['Active_Status'].astype(str)
customer_engagement_dimension['Number_of_Clicks'] = customer_engagement_dimension['Number_of_Clicks'].astype(str)
customer_engagement_dimension['Average_Day_Between_Purchase'] = customer_engagement_dimension['Average_Day_Between_Purchase'].astype(str)

/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/1598328307.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_engagement_dimension['Customer_ID'] = customer_engagement_dimension['Customer_ID'].astype(str)
/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/1598328307.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_engagement_dimension['Active_Status'] = customer_engagement_dimension['Active_Status'].astype(str)
/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000

In [46]:
#customer_engagement_dimension 
def insert_records(recordList):
    insert_record_query3 = """ INSERT INTO customer_engagement_dimension (Customer_ID,Most_frequent_purchased_product,Most_frequent_purchased_category,Coupon_used,Customer_purchasing_power,Active_Status,Number_of_Clicks,Coversion_rate,Average_Day_Between_Purchase) VALUES (?, ?,?,?,?,?,?,?,?)"""
    cursor.executemany(insert_record_query3, recordList)
    conn.commit()
    print('New data inserted successfully')

In [47]:
#Insert dimensions in a list
insert_list = customer_engagement_dimension.to_records(index=False)
insert_records(insert_list) 

New data inserted successfully


In [48]:
#Create Shopping Cart dimension
create_shopping_cart_dimension = """ CREATE TABLE  shopping_cart_dimension 
(Customer_ID TEXT PRIMARY KEY,
Number_of_items_in_cart NUMERIC NOT NULL,
Product_ID_in_cart TEXT NOT NULL,
Number_of_item_checked_out NUMERIC NOT NULL)"""
cursor.execute(create_shopping_cart_dimension)
conn.commit()

In [49]:
#Add dimensions
shopping_cart_dimension = Profitability[['Customer_ID','Number_of_items_in_cart','Product_ID_in_cart','Number_of_item_checked_out']]

In [50]:
#Change data types
shopping_cart_dimension['Customer_ID'] = shopping_cart_dimension['Customer_ID'].astype(str)
shopping_cart_dimension['Number_of_items_in_cart'] = shopping_cart_dimension['Number_of_items_in_cart'].astype(str)
shopping_cart_dimension['Product_ID_in_cart'] = shopping_cart_dimension['Product_ID_in_cart'].astype(str)
shopping_cart_dimension['Number_of_item_checked_out'] = shopping_cart_dimension['Number_of_item_checked_out'].astype(str)

/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/4145377805.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shopping_cart_dimension['Customer_ID'] = shopping_cart_dimension['Customer_ID'].astype(str)
/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/4145377805.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shopping_cart_dimension['Number_of_items_in_cart'] = shopping_cart_dimension['Number_of_items_in_cart'].astype(str)
/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T

In [51]:
#Carrier Shopping Cart dimension
def insert_records(recordList):
    insert_record_query5 = """ INSERT INTO shopping_cart_dimension (Customer_ID,Number_of_items_in_cart,Product_ID_in_cart,Number_of_item_checked_out) VALUES (?, ?,?,?)"""
    cursor.executemany(insert_record_query5, recordList)
    conn.commit()
    print('New data inserted successfully') 

In [52]:
#Add dimensions to a list
insert_list = shopping_cart_dimension.to_records(index=False)
insert_records(insert_list) 

New data inserted successfully


In [53]:
#Create Discount dimension
create_discount_dimension= """ CREATE TABLE  discount_dimension
(Discount_ID TEXT PRIMARY KEY,
Discount_Name TEXT NOT NULL,
Discount_Type TEXT NOT NULL,
Price_treatment_type TEXT NOT NULL,
Coupon_code TEXT NOT NULL,
Discount_Cost Numberic NOT NULL,
Discount_begin_date Varchar(20) NOT NULL,
Discount_end_date Varchar(20) NOT NULL)"""

cursor.execute(create_discount_dimension)
conn.commit()

In [54]:
#Add dimensions
discount_dimension = Profitability[['Discount_ID','Discount_Name','Discount_Type','Price_treatment_type','Coupon_code','Discount_Cost','Discount_begin_date','Discount_end_date']]

In [55]:
#Change data types
discount_dimension['Discount_ID'] = discount_dimension['Discount_ID'].astype(str)
discount_dimension['Discount_Cost'] = discount_dimension['Discount_Cost'].astype(str)
discount_dimension['Discount_begin_date'] = discount_dimension['Discount_begin_date'].astype(str)
discount_dimension['Discount_end_date'] = discount_dimension['Discount_end_date'].astype(str)

/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/3220361566.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  discount_dimension['Discount_ID'] = discount_dimension['Discount_ID'].astype(str)
/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/3220361566.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  discount_dimension['Discount_Cost'] = discount_dimension['Discount_Cost'].astype(str)
/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/3220361566.py:4: Settin

In [56]:
#Insert records
def insert_records(recordList):
    insert_record_query6 = """ INSERT INTO discount_dimension (Discount_ID,Discount_Name,Discount_Type,Price_treatment_type,Coupon_code,Discount_Cost,Discount_begin_date,Discount_end_date) VALUES (?, ?,?,?,?,?,?,?)"""
    cursor.executemany(insert_record_query6, recordList)
    conn.commit()
    print('New data inserted successfully') 

In [57]:
#Add dimensions in a list
insert_list = discount_dimension.to_records(index=False)
insert_records(insert_list) 

New data inserted successfully


In [58]:
#Create Advertising dimension
create_advertising_dimension = """ CREATE TABLE advertising_dimension
(Advertising_ID TEXT PRIMARY KEY,
Ad_Name TEXT NOT NULL,
Ad_Type TEXT NOT NULL,
Advertisement_channel TEXT NOT NULL,
Ad_cost VARCHAR(20) NOT NULL,
Advertising_begin_date VARCHAR(20) NOT NULL,
Advertising_end_date VARCHAR(20) NOT NULL)"""

cursor.execute(create_advertising_dimension)
conn.commit()

In [59]:
#Add dimensions
advertising_dimension = Profitability[['Advertising_ID','Ad_Name','Ad_Type','Advertisement_channel','Ad_cost','Advertising_begin_date','Advertising_end_date']]

In [60]:
#Change data types
advertising_dimension['Advertising_ID'] = advertising_dimension['Advertising_ID'].astype(str) 
advertising_dimension['Ad_cost'] = advertising_dimension['Ad_cost'].astype(str) 
advertising_dimension['Advertising_begin_date'] = advertising_dimension['Advertising_begin_date'].astype(str) 
advertising_dimension['Advertising_end_date'] = advertising_dimension['Advertising_end_date'].astype(str)

/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/697535991.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  advertising_dimension['Advertising_ID'] = advertising_dimension['Advertising_ID'].astype(str)
/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/697535991.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  advertising_dimension['Ad_cost'] = advertising_dimension['Ad_cost'].astype(str)
/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/697535991.py:4: Set

In [61]:
#Insert record values
def insert_records(recordList):
    insert_record_query7 = """ INSERT INTO advertising_dimension (Advertising_ID,Ad_Name,Ad_Type,Advertisement_channel,Ad_cost,Advertising_begin_date,Advertising_end_date) VALUES (?, ?,?,?,?,?,?)"""
    cursor.executemany(insert_record_query7, recordList)
    conn.commit()
    print('New data inserted successfully')

In [62]:
#Add records
insert_list = advertising_dimension.to_records(index=False)
insert_records(insert_list) 

New data inserted successfully


In [63]:
#Create Transaction dimension
create_transaction_dimension = """ CREATE TABLE transaction_dimension
(Transaction_ID TEXT PRIMARY KEY,
Order_ID TEXT NOT NULL,
POS TEXT NOT NULL,
Payment_Type TEXT NOT NULL,
Customer_ID TEXT NOT NULL,
Currency_ID TEXT NOT NULL)"""
cursor.execute(create_transaction_dimension)
conn.commit()

In [64]:
#Add dimensions
transaction_dimension = Profitability[['Transaction_ID','Order_ID','POS','Payment_Type','Customer_ID', 'Currency_ID']]

In [65]:
#Change data types
transaction_dimension['Transaction_ID'] = transaction_dimension['Transaction_ID'].astype(str)
transaction_dimension['Currency_ID'] = transaction_dimension['Currency_ID'].astype(str)

/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/20130665.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transaction_dimension['Transaction_ID'] = transaction_dimension['Transaction_ID'].astype(str)
/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/20130665.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transaction_dimension['Currency_ID'] = transaction_dimension['Currency_ID'].astype(str)


In [66]:
#Insert records
def insert_records(recordList):
    insert_record_query4 = """ INSERT INTO transaction_dimension (Transaction_ID,Order_ID,POS,Payment_Type,Customer_ID, Currency_ID) VALUES (?, ?,?,?,?,?)"""
    cursor.executemany(insert_record_query4, recordList)
    conn.commit()
    print('New data inserted successfully') 

In [67]:
#Add data values
insert_list = transaction_dimension.to_records(index=False)
insert_records(insert_list) 

New data inserted successfully


In [68]:
#Insert Customer Information dimension
create_customer_information_dimension = """ CREATE TABLE  customer_information_dimension
(Customer_ID TEXT PRIMARY KEY,
Customer_engagement_active_status TEXT NOT NULL,
Number_of_transactions Varchar NOT NULL,
Home_Address TEXT NOT NULL,
Email TEXT NOT NULL,
Phone_Number Varchar(20) NOT NULL,
Account_Number VARCHAR(20) NOT NULL)"""

In [69]:
#Add dimensions
cursor.execute(create_customer_information_dimension)
conn.commit()

In [70]:
#Change data types
customer_information_dimension = Profitability[['Customer_ID', 'Number_of_transactions','Customer_engagement_active_status','Home_Address','Email','Phone_Number','Account_Number']]
customer_information_dimension['Customer_ID'] = customer_information_dimension['Customer_ID'].astype(str)
customer_information_dimension['Number_of_transactions'] = customer_information_dimension['Number_of_transactions'].astype(str)
customer_information_dimension['Phone_Number'] = customer_information_dimension['Phone_Number'].astype(str)

/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/14925670.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_information_dimension['Customer_ID'] = customer_information_dimension['Customer_ID'].astype(str)
/var/folders/hz/ykyh731j4wvc21cj6rsxtyy80000gn/T/ipykernel_21716/14925670.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_information_dimension['Number_of_transactions'] = customer_information_dimension['Number_of_transactions'].astype(str)
/var/folders/hz/ykyh731j4w

In [71]:
#Insert records
def insert_records(recordList):
    insert_record_query8 = """ INSERT INTO customer_information_dimension (Customer_ID, Number_of_transactions,Customer_engagement_active_status,Home_Address,Email,Phone_Number,Account_Number) VALUES (?, ?,?,?,?,?,?)"""
    cursor.executemany(insert_record_query8, recordList)
    conn.commit()
    print('New data inserted successfully')

In [72]:
#Add data values
insert_list = customer_information_dimension.to_records(index=False)
insert_records(insert_list) 

New data inserted successfully


In [77]:
#Create Product dimension
create_product_dimension = """ CREATE TABLE  product_dimension
(Product_ID TEXT PRIMARY KEY,
Product_Name TEXT NOT NULL,
Product_Size TEXT NOT NULL,
Product_Type TEXT NOT NULL,
Product_Color TEXT NOT NULL,
Product_Weight TEXT NOT NULL,
Brand TEXT NOT NULL,
Unit_per_shipping_case NUMERIC NOT NULL,
Manufacture_name TEXT NOT NULL,
Department_name TEXT NOT NULL,
Seller_ID Varchar(20) NOT NULL,
Category_Name TEXT NOT NULL,
Unit_Price TEXT NOT NULL)"""
cursor.execute(create_product_dimension)
conn.commit()

In [81]:
#Change data types
product_dimension = Profitability[['Product_Name', 'Product_ID', 'Product_Size', 'Product_Type', 'Product_Color', 'Product_Weight', 'Brand', 'Unit_per_shipping_case', 'Manufacture_name', 'Department_name', 'Category_Name', 'Seller_ID','Unit_Price']]
product_dimension['Product_ID'] = product_dimension['Product_ID'].astype(str)
product_dimension['Unit_per_shipping_case'] = product_dimension['Unit_per_shipping_case'].astype(str)
product_dimension['Seller_ID'] = product_dimension['Seller_ID'].astype(str)

KeyError: "['Unit_Price'] not in index"

In [ ]:
#Insert records
def insert_records(recordList):
    insert_record_query9 = """ INSERT INTO product_dimension (Product_Name, Product_ID, Product_Size, Product_Type, Product_Color, Product_Weight, Brand, Unit_per_shipping_case, Manufacture_name, Department_name, Category_Name, Seller_ID,Unit_price) VALUES (?,?, ?,?,?,?,?,?,?,?,?,?,?)"""
    cursor.executemany(insert_record_query9, recordList)
    conn.commit()
    print('New data inserted successfully')

In [ ]:
#Add data values
insert_list = product_dimension.to_records(index=False)
insert_records(insert_list) 

In [ ]:
#Create Seller dimension table
create_seller_dimension = """ CREATE TABLE  seller_dimension
(Seller_ID TEXT PRIMARY KEY,
Seller_Name TEXT NOT NULL,
Seller_Type TEXT NOT NULL,
Seller_City TEXT NOT NULL,
Seller_Country TEXT NOT NULL,
Seller_Province TEXT NOT NULL,
Seller_Street TEXT NOT NULL,
Last_Name TEXT NOT NULL,
First_Name TEXT NOT NULL,
Contact_phone_country_code TEXT NOT NULL,
PHONE_NUMBER TEXT NOT NULL)"""
cursor.execute(create_seller_dimension)
conn.commit()

In [ ]:
#Change data types
seller_dimension = Profitability[['Seller_ID', 'Seller_Name', 'Seller_Type', 'Seller_City', 'Seller_Province', 'Seller_Country', 'Seller_Street', 'Last_Name', 'First_Name', 'Contact_phone_country_code', 'PHONE_NUMBER']]
seller_dimension['Seller_ID'] = seller_dimension['Seller_ID'].astype(str) 
seller_dimension['Contact_phone_country_code'] = seller_dimension['Contact_phone_country_code'].astype(str) 
seller_dimension['PHONE_NUMBER'] = seller_dimension['PHONE_NUMBER'].astype(str) 

In [ ]:
#Insert records
def insert_records(recordList):
    insert_record_query9 = """ INSERT INTO seller_dimension (Seller_ID, Seller_Name, Seller_Type, Seller_City, Seller_Province, Seller_Country, Seller_Street, Last_Name, First_Name, Contact_phone_country_code, PHONE_NUMBER) VALUES (?, ?,?,?,?,?, ?,?,?,?,?)"""
    cursor.executemany(insert_record_query9, recordList)
    conn.commit()
    print('New data inserted successfully')

In [ ]:
#Add data values
insert_list = seller_dimension.to_records(index=False)
insert_records(insert_list) 

In [ ]:
#Create Profitability Fact Table & specify data types and constraints
create_table_query2 = """ CREATE TABLE  Profitability_Fact_Table
(Product_ID Varchar(20) PRIMARY KEY,
Date_ID NUMERIC NOT NULL,
Transaction_ID TEXT NOT NULL,
Discount_ID TEXT NOT NULL,
Advertising_ID TEXT NOT NULL,  
Customer_ID TEXT NOT NULL)"""
#Execute query and make change permanent
cursor.execute(create_table_query2)
conn.commit()

In [ ]:
#Add features to fact table
Profitability_Fact_Table = Profitability[["Product_ID",'Date_ID','Transaction_ID','Discount_ID','Advertising_ID','Customer_ID']]

In [ ]:
#Change data type
Profitability_Fact_Table["Product_ID"] = Profitability_Fact_Table["Product_ID"].astype(str) 
Profitability_Fact_Table['Date_ID'] = Profitability_Fact_Table['Date_ID'].astype(str)

In [ ]:
#Insert records
def insert_records(recordList):
    insert_record_query9 = """ INSERT INTO Profitability_Fact_Table (Product_ID,Date_ID,Transaction_ID,Discount_ID,Advertising_ID,Customer_ID) VALUES (?, ?,?,?,?,?)"""
    cursor.executemany(insert_record_query9, recordList)
    conn.commit()
    print('New data inserted successfully')

In [ ]:
#Add data values
insert_list = Profitability_Fact_Table.to_records(index=False)
insert_records(insert_list) 